In [1]:
state = {
    "stage": "conversation",   
    "intent": None,
    "name": None,
    "email": None,
    "platform": None,
    "history": []
}


In [2]:
def detect_intent(message: str) -> str:
    msg = message.lower()

    # HIGH-INTENT FIRST 
    if any(word in msg for word in ["i want", "sign up", "try", "get started", "buy"]):
        return "high_intent"

    # Pricing-only intent
    if any(word in msg for word in ["price", "pricing", "cost"]):
        return "pricing_query"

    return "casual"


In [3]:
def answer_with_rag(query: str) -> str:
    if "pro" in query.lower():
        return (
            "Our Pro plan costs $79/month and includes unlimited videos, "
            "4K resolution, and AI captions."
        )

    return (
        "We offer a Basic plan at $29/month (10 videos, 720p) "
        "and a Pro plan at $79/month with advanced features."
    )


In [4]:
def mock_lead_capture(name: str, email: str, platform: str):
    print(f" Lead captured successfully: {name}, {email}, {platform}")


In [5]:
def agent_step(user_input: str, state: dict) -> dict:

    # ================= STAGE 1: CONVERSATION =================
    if state["stage"] == "conversation":

        state["history"].append(f"User: {user_input}")
        intent = detect_intent(user_input)

        # Pricing flow
        if intent == "pricing_query":
            state["intent"] = "pricing_query"
            reply = answer_with_rag(user_input)
            state["history"].append(f"Agent: {reply}")
            return state

        # High-intent → lock into lead capture
        if intent == "high_intent":
            state["intent"] = "high_intent"
            state["stage"] = "lead_capture"
            state["history"].append("Agent: Great! May I have your name?")
            return state

        # Default reply
        state["history"].append("Agent: How can I help you today?")
        return state

    # ================= STAGE 2: LEAD CAPTURE =================
    if state["stage"] == "lead_capture":

        state["history"].append(f"User: {user_input}")

        if state["name"] is None:
            state["name"] = user_input
            state["history"].append("Agent: Thanks! Please share your email.")
            return state

        if state["email"] is None:
            state["email"] = user_input
            state["history"].append(
                "Agent: Which platform do you create content on?"
            )
            return state

        if state["platform"] is None:
            state["platform"] = user_input

            #  TOOL EXECUTION 
            mock_lead_capture(
                state["name"],
                state["email"],
                state["platform"]
            )

            state["history"].append(
                "Agent: Thanks! Our team will contact you shortly "
            )
            return state

    return state


In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    state = agent_step(user_input, state)
    print(state["history"][-1])


User:  hi


Agent: How can I help you today?


User:  tell me about pricing


Agent: We offer a Basic plan at $29/month (10 videos, 720p) and a Pro plan at $79/month with advanced features.


User:  i want the pro plan


...

Agent: Thanks! Please share your email.


User:  ankit@gamil.com


Agent: Which platform do you create content on?


User:  youtube


 Lead captured successfully: ankit, ankit@gamil.com, youtube
Agent: Thanks! Our team will contact you shortly 
